In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [37]:
BASE_MODEL = "google/gemma-2b-it"
ADAPTER_MODEL = "outputs/checkpoint-50"  # 학습 후 모델저장 경로

In [38]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

## 학습 전 모델 테스트

In [39]:
model_origin = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:

# 추론 모드
model_origin.eval()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

# 테스트 문서
document = """
서울시는 29일 2025년 예산안을 발표했다. 총 예산 규모는 50조원으로 
전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 
복지 정책 강화다. 시는 지하철 노선 확장과 저소득층 지원 예산을 
대폭 늘렸다고 밝혔다.
"""

# 프롬프트 생성 (tokenizer가 <bos>를 자동으로 추가하므로 제거)
prompt = f"""<start_of_turn>user
다음 글을 요약해주세요:

{document.strip()}<end_of_turn>
<start_of_turn>model
"""

# 토크나이징
inputs = tokenizer(prompt, return_tensors="pt").to(next(model_origin.parameters()).device)

# 생성
print("\n생성 중...")
with torch.no_grad():
    outputs = model_origin.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,  # 더 낮은 temperature로 일관성 향상
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,  # 반복 방지
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# 결과 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("\n=== 전체 출력 ===")
print(result)

# 요약만 추출 (더 정확한 파싱)
if "<start_of_turn>model" in result:
    summary = result.split("<start_of_turn>model")[-1]
    # <end_of_turn>이나 <eos> 전까지만 추출
    summary = summary.split("<end_of_turn>")[0].split("<eos>")[0].strip()
    
    # 이상한 패턴 제거
    patterns_to_remove = ["municipi :", "qed", "QED", "Q.E.D."]
    for pattern in patterns_to_remove:
        if pattern in summary:
            summary = summary.split(pattern)[0].strip()
    
    # 끝에 붙은 이상한 단어들 제거
    summary = summary.rstrip(" .").strip()
else:
    summary = result.strip()

print("\n=== 요약 결과 ===")
print(summary)


생성 중...

=== 전체 출력 ===
<bos><start_of_turn>user
다음 글을 요약해주세요:

서울시는 29일 2025년 예산안을 발표했다. 총 예산 규모는 50조원으로 
전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 
복지 정책 강화다. 시는 지하철 노선 확장과 저소득층 지원 예산을 
대폭 늘렸다고 밝혔다.<end_of_turn>
<start_of_turn>model
* 서울시는 2025년 예산안을 발표했다.
* 총 예산 규모는 50조원으로 5% 증가했다.
* 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다.
* 시는 지하철 노선 확장과 저소득층 지원 예산을 늘렸다고 밝혔다.<eos>

=== 요약 결과 ===
* 서울시는 2025년 예산안을 발표했다.
* 총 예산 규모는 50조원으로 5% 증가했다.
* 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다.
* 시는 지하철 노선 확장과 저소득층 지원 예산을 늘렸다고 밝혔다


## 파인튜닝 후 LoRA 어뎁터 적용 테스트

In [41]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

model = PeftModel.from_pretrained(model, ADAPTER_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:

# 추론 모드
model.eval()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

# 테스트 문서
document = """
서울시는 29일 2025년 예산안을 발표했다. 총 예산 규모는 50조원으로 
전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 
복지 정책 강화다. 시는 지하철 노선 확장과 저소득층 지원 예산을 
대폭 늘렸다고 밝혔다.
"""

# 프롬프트 생성 (tokenizer가 <bos>를 자동으로 추가하므로 제거)
prompt = f"""<start_of_turn>user
다음 글을 요약해주세요:

{document.strip()}<end_of_turn>
<start_of_turn>model
"""

# 토크나이징
inputs = tokenizer(prompt, return_tensors="pt").to(next(model.parameters()).device)

# 생성
print("\n생성 중...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,  # 더 낮은 temperature로 일관성 향상
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,  # 반복 방지
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# 결과 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("\n=== 전체 출력 ===")
print(result)

# 요약만 추출 (더 정확한 파싱)
if "<start_of_turn>model" in result:
    summary = result.split("<start_of_turn>model")[-1]
    # <end_of_turn>이나 <eos> 전까지만 추출
    summary = summary.split("<end_of_turn>")[0].split("<eos>")[0].strip()
    
    # 이상한 패턴 제거
    patterns_to_remove = ["municipi :", "qed", "QED", "Q.E.D."]
    for pattern in patterns_to_remove:
        if pattern in summary:
            summary = summary.split(pattern)[0].strip()
    
    # 끝에 붙은 이상한 단어들 제거
    summary = summary.rstrip(" .").strip()
else:
    summary = result.strip()

print("\n=== 요약 결과 ===")
print(summary)


생성 중...

=== 전체 출력 ===
<bos><start_of_turn>user
다음 글을 요약해주세요:

서울시는 29일 2025년 예산안을 발표했다. 총 예산 규모는 50조원으로 
전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 
복지 정책 강화다. 시는 지하철 노선 확장과 저소득층 지원 예산을 
대폭 늘렸다고 밝혔다.<end_of_turn>
<start_of_turn>model
서울시는 29일 2025년 예산안을 발표했는데, 총 예산 규모는 50조원으로 전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다. 서울시는 지하철 노선 확장과 저소득층 지원 예산을 늘렸다고 밝혔다.municipi : 서울시는 29일 2025년 예산안을 발표했는데, 총 예산 규모는 50조원으로 전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다. 서울시는 지하철 노선 확장과 저소득층 지원 예산을 늘렸다고 밝혔다.municipi : 서울시

=== 요약 결과 ===
서울시는 29일 2025년 예산안을 발표했는데, 총 예산 규모는 50조원으로 전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다. 서울시는 지하철 노선 확장과 저소득층 지원 예산을 늘렸다고 밝혔다


In [43]:
model_origin = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
# 모델 로드
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(model, ADAPTER_MODEL)

# 추론 모드
model.eval()

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

# 테스트 문서
document = """
서울시는 29일 2025년 예산안을 발표했다. 총 예산 규모는 50조원으로 
전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 
복지 정책 강화다. 시는 지하철 노선 확장과 저소득층 지원 예산을 
대폭 늘렸다고 밝혔다.
"""

# 프롬프트 생성 (tokenizer가 <bos>를 자동으로 추가하므로 제거)
prompt = f"""<start_of_turn>user
다음 글을 요약해주세요:

{document.strip()}<end_of_turn>
<start_of_turn>model
"""

# 토크나이징
inputs = tokenizer(prompt, return_tensors="pt").to(next(model.parameters()).device)

# 생성
print("\n생성 중...")
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3,  # 더 낮은 temperature로 일관성 향상
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.2,  # 반복 방지
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

# 결과 출력
result = tokenizer.decode(outputs[0], skip_special_tokens=False)
print("\n=== 전체 출력 ===")
print(result)

# 요약만 추출 (더 정확한 파싱)
if "<start_of_turn>model" in result:
    summary = result.split("<start_of_turn>model")[-1]
    # <end_of_turn>이나 <eos> 전까지만 추출
    summary = summary.split("<end_of_turn>")[0].split("<eos>")[0].strip()
    
    # 이상한 패턴 제거
    patterns_to_remove = ["municipi :", "qed", "QED", "Q.E.D."]
    for pattern in patterns_to_remove:
        if pattern in summary:
            summary = summary.split(pattern)[0].strip()
    
    # 끝에 붙은 이상한 단어들 제거
    summary = summary.rstrip(" .").strip()
else:
    summary = result.strip()

print("\n=== 요약 결과 ===")
print(summary)

Loading base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LoRA adapter...

생성 중...

=== 전체 출력 ===
<bos><start_of_turn>user
다음 글을 요약해주세요:

서울시는 29일 2025년 예산안을 발표했다. 총 예산 규모는 50조원으로 
전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 
복지 정책 강화다. 시는 지하철 노선 확장과 저소득층 지원 예산을 
대폭 늘렸다고 밝혔다.<end_of_turn>
<start_of_turn>model
서울시는 29일 2025년 예산안을 발표했는데, 총 예산 규모는 50조원으로 전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다. 서울은 지하철 노선 확장과 저소득층 지원 예산을 대폭 늘렸다고 밝혔다.municipi : 서울시의 2025년 예산안이 발표됐다. seoulnews1.com 김영진 기자 서울시는 지하철 노선 확장과 
복지 정책 강화를 위해 29일 총 예산 규모 50조원을 발표했다. 서울시는 지하철 노선 확장과 저소득층 지원 예산을 대폭 늘렸다고 밝혔다. 서울

=== 요약 결과 ===
서울시는 29일 2025년 예산안을 발표했는데, 총 예산 규모는 50조원으로 전년 대비 5% 증가했다. 주요 투자 분야는 교통 인프라 확충과 복지 정책 강화다. 서울은 지하철 노선 확장과 저소득층 지원 예산을 대폭 늘렸다고 밝혔다
